# Enrolments Data

## Exploration
### Statistical exploration

#### Reading the data, removing useless columns and adding special columns

In [2]:
import pandas as pd
import numpy as np
#Read the data
enrolments = pd.read_csv("data/courses snap_sales_funnel 2017-06-27T1636.csv")

In [3]:
enrolments.columns

Index(['Startdate', 'Enddate', 'Price', 'Metro', 'Course', 'Type', 'LclPrice',
       'DaysEnroltoStartdate', 'DaysInvoicetoEnrol', 'DaysLeadtoEnroled',
       'EnrolDate', 'CanceledDate', 'Email', 'ExpectedPayment',
       'InvoiceAmountLcl', 'ApplicationType', 'PardotCategory', 'Status',
       'Email.1', 'LeadID', 'OppCreatedDate', 'OppOwnerMetro', 'OppOwnerEmail',
       'PhoneValid', 'SpeedtoLead', 'CampaignSource', 'CampaignSourceCategory',
       'InstanceID'],
      dtype='object')

In [4]:
enrolments.PhoneValid.unique()
enrolments['PhoneValid'] = enrolments.PhoneValid.map({'Yes': 1,'No': 0})
#Remove some useless columns
enrolments.drop('LclPrice', axis=1, inplace=True)
enrolments.drop('Email', axis=1, inplace=True)
enrolments.drop('Status', axis=1, inplace=True)
enrolments.drop('CampaignSourceCategory',axis=1,inplace=True)
enrolments.drop('CampaignSource',axis=1,inplace=True)
enrolments.drop('OppOwnerMetro', axis=1, inplace=True)
enrolments.drop('OppOwnerEmail', axis=1, inplace=True)
enrolments.drop('OppCreatedDate', axis=1, inplace=True)
enrolments.drop('InvoiceAmountLcl', axis=1, inplace=True)
enrolments.drop('Email.1', axis=1, inplace=True)
enrolments.columns

Index(['Startdate', 'Enddate', 'Price', 'Metro', 'Course', 'Type',
       'DaysEnroltoStartdate', 'DaysInvoicetoEnrol', 'DaysLeadtoEnroled',
       'EnrolDate', 'CanceledDate', 'ExpectedPayment', 'ApplicationType',
       'PardotCategory', 'LeadID', 'PhoneValid', 'SpeedtoLead', 'InstanceID'],
      dtype='object')

In [5]:
#Add cancelled column to track whether an enrolment cancelled
enrolments['Cancelled'] = enrolments.CanceledDate.notnull()

In [ ]:
enrolments.drop('CanceledDate', axis=1, inplace=True)
enrolments.columns

In [ ]:
#Add an APAC column
enrolments['APAC'] = enrolments['Metro'].isin(['sydney','melbourne','hong-kong','singapore','brisbane'])
enrolments.APAC.value_counts()

In [ ]:
#Limit to only post mid-2014 data
enrolments = enrolments[enrolments.Startdate > 20140630]

In [ ]:
#Checkout all the numeric columns
#enrolments.info()
#enrolments.describe()
enrolments.shape

In [ ]:
enrolments.info()

#### Dealing with null values

In [ ]:
enrolments.isnull().sum()

In [ ]:
enrolments['ExpectedPayment'] = enrolments.ExpectedPayment.map({
    'Student / Self': 'Student / Self',
    'Student (self)': 'Student / Self',
    'Employer': 'Employer',
    'Loan (Partner, please note)': 'Loan Partner',
    'Veterans Affairs': 'Veterans Affairs',
    'Employer Bulk Pre-paid': 'Employer',
    'Scholarship (Opportunity Fund)': 'Scholarship',
    'Loan Partner (please note)': 'Loan Partner',
    'Family':'Family',
    'Other (please note)':'Other',
    'GA Employee/Instructor':'GA Employee/Instructor',
    'Loan (Private Bank, please note)':'Loan (Private)',
    'Government Funding / Grant':'Government Funding / Grant',
    'Other':'Other',
    'CourseHorse':'Other',
    'Opportunity Fund':'Scholarship',
    'Enterprise Credit':'Other',
    'Scholarship (GA Partner, please note)':'Scholarship',
    'Scholarship (Other, please note)':'Scholarship',
})
enrolments.ExpectedPayment.fillna("Unknown", inplace=True)
enrolments.ExpectedPayment.value_counts()

In [ ]:
enrolments.PardotCategory.fillna("Unknown", inplace=True)
enrolments.PardotCategory.value_counts()

In [ ]:
#enrolments.DaysEnroltoStartdate.mean()
enrolments.DaysEnroltoStartdate.fillna(enrolments.DaysEnroltoStartdate.mean(), inplace=True)

In [ ]:
#enrolments.SpeedtoLead.value_counts()
enrolments.SpeedtoLead.fillna(enrolments.SpeedtoLead.mean(), inplace=True)

In [ ]:
enrolments.isnull().sum()

#### Streamlining multiple categories

In [ ]:
#Checkout only the more common application types
#enrolments.ApplicationType.value_counts()<100

In [ ]:
commonapptypes = enrolments.ApplicationType.value_counts().index[enrolments.ApplicationType.value_counts()>147]
commonapptypes

In [ ]:
enrolments = enrolments[enrolments.ApplicationType.isin(commonapptypes)]
enrolments.shape

In [ ]:
enrolments.ApplicationType.value_counts()

In [ ]:
#Checkout only the more common Pardot types
#enrolments.PardotCategory.value_counts()>100
commonpdtypes = enrolments.PardotCategory.value_counts().index[enrolments.PardotCategory.value_counts()>100]
commonpdtypes

In [ ]:
enrolments = enrolments[enrolments.PardotCategory.isin(commonpdtypes)]
enrolments.shape

In [ ]:
enrolments.PardotCategory.value_counts()

In [ ]:
enrolments.info()

In [ ]:
enrolments.Cancelled.value_counts()

#### Pandas profile

In [ ]:
#!pip install pandas-profiling

In [ ]:
#import pandas_profiling
#apandas_profiling.ProfileReport(enrolments)

### Visualisation

In [ ]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
import plotly
import plotly.plotly as py
import plotly.graph_objs as go 
import folium
import plotly.figure_factory as ff

plotly.tools.set_credentials_file(username='Msquirchuk', api_key='kTj4gydNrsbGuMDjMcNn')

%matplotlib inline
sns.set(color_codes=True)

In [ ]:
enrolments['DaysInvoicetoEnrol'].hist()

In [ ]:
enrolments['DaysEnroltoStartdate'].hist()

In [ ]:
plt.figure(figsize = (15,6))
sns.barplot(x='Metro', y = 'Cancelled', data = enrolments)

In [ ]:
plt.figure(figsize = (20,10))
sns.barplot(x='Course', y = 'Cancelled', data = enrolments)

In [ ]:
sns.jointplot(x='DaysEnroltoStartdate', y="DaysInvoicetoEnrol", data=enrolments)

In [ ]:
corr = enrolments.corr()
corr = (corr)
plt.figure(figsize = (20,20))
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
sns.plt.title('Heatmap of Correlation Matrix')

## Running dummy data
- These columns have text outputs:
 - Metro: 35282 non-null object
 - Course: 35282 non-null object
 - Type: 35282 non-null object
 - ExpectedPayment: 35282 non-null object
 - ApplicationType: 35282 non-null object
 - PardotCategory: 35282 non-null object

In [ ]:
dummydata = pd.get_dummies(data=enrolments, columns = ['Metro','Course','Type','ExpectedPayment','ApplicationType','PardotCategory'], prefix = ['Metro','Course','Type','ExpectedPayment','ApplicationType','PardotCategory'] )

In [ ]:
dummydata.columns

In [ ]:
enrolments.columns

## Let's try some algorithms!

### Decision Tree

In [ ]:
dummydata.Cancelled.value_counts()

In [ ]:
treesubset = dummydata

In [ ]:
from sklearn import tree
from sklearn.cross_validation import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt

%matplotlib inline

cancelled = treesubset['Cancelled']
del treesubset['Cancelled']
treesubset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(treesubset, cancelled, random_state=1)
ctree = tree.DecisionTreeClassifier(random_state=1, max_depth=2)
# Fit the decision tree classifier
ctree.fit(X_train, y_train)
# Create a feature vector
features = treesubset.columns.tolist()
features

In [ ]:
from io import StringIO
out = StringIO()
tree.export_graphviz(ctree, out_file = out)
from sklearn.tree import export_graphviz
export_graphviz(ctree, out_file='decisiontree.dot', feature_names=features)

#### Previously exported: 
<img src="decisiontree-enrolments.png">

### Logistic Regression

In [ ]:
import pandas as pd
# data = dummydata
all_ftcolumns = dummydata.columns.tolist()
all_ftcolumns.remove('Cancelled')

In [ ]:
X = dummydata[all_ftcolumns]
y = dummydata.Cancelled

In [ ]:
#split the data
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
#fit the logreg model and print the coefficients
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
zip(feature_cols, logreg.coef_[0])
print(logreg.fit(X_train, y_train))

In [ ]:
#test accuracy
y_pred_class = logreg.predict(X_test)
from sklearn import metrics
print(metrics.accuracy_score(y_test,y_pred_class))

In [ ]:
from sklearn.dummy import DummyClassifier
dumb = DummyClassifier(strategy='most_frequent')
dumb.fit(X_train, y_train)
y_dumb_class = dumb.predict(X_test)
print (metrics.accuracy_score(y_test, y_dumb_class))

In [ ]:
scores = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=10)
print(scores)

In [ ]:
print(scores.mean())

In [ ]:
#print the confusion matrix
from sklearn import metrics
prds = logreg.predict(X)
print(metrics.confusion_matrix(y_test, y_pred_class))

In [ ]:
#generate the ROC curve
import matplotlib.pyplot as plt
# Generate the prediction values for each of the test observations using predict_proba() function rather than just predict
preds = logreg.predict_proba(X_test)[:,1]
# Store the false positive rate(fpr), true positive rate (tpr) in vectors for use in the graph
fpr, tpr, _ = metrics.roc_curve(y_test, preds)
# Store the Area Under the Curve (AUC) so we can annotate our graph with this metric
roc_auc = metrics.auc(fpr,tpr)
# Plot the ROC Curve
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()
# Our aim when modelling is to maximise the area under the curve, the closer to one the better the model.

### Regression 

In [ ]:
print('RMSE (No reg.) =', np.sqrt(metrics.mean_squared_error(y_test,y_pred_class)))

In [ ]:
from sklearn.linear_model import RidgeCV
alpha_range = 10.**np.arange(-5, 5)
rregcv = RidgeCV(normalize=True, scoring='neg_mean_squared_error', alphas=alpha_range)
rregcv.fit(X_train, y_train)
# Print the optimal value of Alpha for Ridge Regression
print('Optimal Alpha Value: ', rregcv.alpha_)
# Print the RMSE for the ridge regression model
preds = rregcv.predict(X_test)
print ('RMSE (Ridge CV reg.) =', np.sqrt(metrics.mean_squared_error(y_test, preds)))

In [ ]:
from sklearn.linear_model import LassoCV
alpha_range = 10.**np.arange(-4, 4)
lascv = LassoCV(normalize=True, alphas=alpha_range)
lascv.fit(X_train, y_train)
print('Optimal Alpha Value: ',lascv.alpha_)
lascv.coef_
preds = lascv.predict(X_test)
print('RMSE (Lasso CV reg.) =', np.sqrt(metrics.mean_squared_error(y_test, preds)))

In [ ]:
from sklearn.linear_model import ElasticNetCV
alpha_range = 10.**np.arange(-3, 3)
enetCV = ElasticNetCV(normalize=True, alphas=alpha_range)
enetCV.fit(X_train, y_train)
print('Optimal Alpha Value: ',enetCV.alpha_)
enetCV.coef_
preds = enetCV.predict(X_test)
print('RMSE (ENET CV reg.)', np.sqrt(metrics.mean_squared_error(y_test, preds)))

### KNN Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN_model = KNeighborsClassifier(5)
KNN_model.fit(X_train, y_train)
y_pred_class = KNN_model.predict(X_test)
# Print the new accuracy rate
print(metrics.accuracy_score(y_test, y_pred_class))

### Random Forrest

In [ ]:
#all_ftcolumns

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfclf = RandomForestClassifier(n_estimators=400, max_features=2, oob_score=True, random_state=1)
rfclf.fit(dummydata[all_ftcolumns], dummydata.Cancelled)
rfclf.oob_score_

In [ ]:
pd.DataFrame({'feature':all_ftcolumns, 'importance':rfclf.feature_importances_})

## Parameter tuning?